
# 1. Introduction

This notebook has been developed for the purposes of the EMPIR project “Metrology for the Factory of the Future” (Met4FoF) -  Activity A1.2.2 of the Work Package 1.Two laboratories in PTB and CEM posses conventional dynamic calibration set-ups for acceleration sensors under test. The objective of the task is to compare measurement results of the two laboratories, PTB and CEM, provided by the extended dynamic calibration systems.  

## 1.1. State of the art

In conventional dynamic calibration procedures for acceleration sensors, the acceleration used for the sensor input is applied either as **a sinusoidal excitation with a given frequency and amplitude** or as **a singular shock-like excitation characterized by pulse width and intensity**. 
<br>The quantity is then measured by a reference sensor and by the device under test (DUT). The results of DUT are compared to the reference and hence characterized and linked to the SI. In a dynamic calibration situation, the response of the DUT to time-varying input is the major interest. Hence, it is crucial that the mechanical input operates simultaneously and equally on the reference and DUT. Both, the reference and the DUT provide **electrical outputs (typical voltage)** while **the data acquisition electronics** of the system provides **the analogue to digital conversion (ADC).**
In order to connect the reference and the DUT to the ADC system, two analogue channels are needed. In order to get reliable information on the time dependent response, the timing of the data acquisition on the two channels has to be synchronized. This is typically accomplished by provision of a common clock signal to drive the sampling units of the ADC and a common trigger to start (or mark) the beginning of the acquisition. 

## 1.2. The extension of conventional dynamic calibration systems to digital-output sensors 

In a set-up where the DUT is a digital-output sensor,the sampling of the DUT time series is no longer under the control (trigger, clock) of the calibration system. Instead, the DUT comprises its own digitizer unit with a time base independent of the calibration system. In fact, a typical conventional calibration set-up does not provide an input for digital data at all.
The solution to this problem requires two extensions:
1.	A digital acquisition unit (DAU) which is capable to connect to the digital interface of the sensor under calibration and store/transmit the DUT time-series for later analysis
2.	An additional synchronization signal that provides the link between the time-base of the calibration system and the time-base of the DUT.

The concept for the extension of existing facilities for dynamic calibration uses a custom digital acquisition unit microcontroller board with a connected reference time signal for traceable time stamping of acquired data points. This allows for a synchronised data acquisition from the reference measurement and the DUT. The selected sensor for acceleration measurements is a three axial low-g acceleration sensor with digital output, which allows measurements of acceleration in three perpendicular axes.**At the moment and in this notebook, only an X-axis acceleration has been observed.** 
    
The selected laboratories posses their set-ups and the calibration items were calibrated in both set-ups.The analysis required transformation from time domain to the frequency domain, where frequencies and corresponding magnitudes and phases were calculated.The measurement conditions were kept according to the laboratory standard conditions. 

Each laboratory submitted HDFT files containing groups:
- EXPERIMENTS - containing 171 files (experiment) corresponding to the sine excitations
- RAWDATA - where data from ADCs (*voltage, absolute time, absolute time uncertainty*)  and sensors (*absolute time, absolute time uncertainty, acceleration, angular velocity, magnetic flux density and temperature*) can be approached during the measurements
- RAWTRANSFERFUNCTION - *this group contains quantities of interest: frequencies, amplitudes, assigned uncertainties of amplitudes, phases,  assigned uncertainties of phases, excitation amplitudes and assigned uncertainties of excitation amplitudes
- REFERENCEDATA - reference data from ADCs and sensors.

    
| <b>PTB<b> | Sensor | Internal ADC |
| --- | --- | --- |
| <b>Name<b> | MPU 9250| STM 32 Internal ADC |
| <b>ID<b>  | 535035904 | 535038464 |
| <b>Quantity<b>  | X Acceleration | Voltage  @CH1 |
| <b>Unit<b>  | $\frac{m}{s^{2}}$ | V |    
| <b>Resolution<b>  | 65536,0 (16-bit) | 4096,0 (12-bit) |
| <b>Min. scale<b>  | -156,91439819335938 $\frac{m}{s^{2}}$  | -10 V |
| <b>Min. scale<b>  | 156,90960693359375 $\frac{m}{s^{2}}$  | 10 V |
    
| <b>CEM<b> | Sensor | Internal ADC |
| --- | --- | --- |
| <b>Name<b> | MPU 9250| STM 32 Internal ADC |
| <b>ID<b>  | 3167420416 | 31674422976 |
| <b>Quantity<b>  | X Acceleration | Voltage  @CH1 |
| <b>Unit<b>  | $\frac{m}{s^{2}}$ | V |    
| <b>Resolution<b>  | 65536,0 (16-bit) | 4096,0 (12-bit) |
| <b>Min. scale<b>  | -156,91439819335938 $\frac{m}{s^{2}}$  | -10 V |
| <b>Min. scale<b>  | 156,90960693359375 $\frac{m}{s^{2}}$  | 10 V |

In [3]:
import h5py
import numpy as np
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import scipy.stats
import numpy as np
import scipy.stats as stats
import seaborn as sns
from ipywidgets import widgets, Layout
from IPython import display


Bad key "text.kerning_factor" on line 4 in
C:\Users\EhlimanaJugo\anaconda3\envs\Ehlimana\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# 2.Extract the data

Data for ILC comparison is extracted from the HDF5 files separately for PTB and CEM. The extracted data will be sorted by frequency at the end of the Notebook and saved into Excel file.

In [4]:
def extract_data(filename, sensor_index):
    #explore the HDF5 file, folders and subfolders
    with h5py.File(filename,'r') as f:
        base_items=list(f.items())
        print("\nItems in directory", base_items)
        rawtransfer=f.get("RAWTRANSFERFUNCTION")
        rawtransfer_items=list(rawtransfer.items())
        print("\nItems in reference", rawtransfer_items)
        subgroup=rawtransfer.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250")
        subgroup_items=list(subgroup.items())
        print("\n"+sensor_index+"_MPU_9250 items:",subgroup_items)
        subgroup_acceleration=subgroup.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration")
        subgroup_acceleration_items=list(subgroup_acceleration.items())
        print("\nAcceleration items:",subgroup_acceleration_items)
        subgroup_acceleration_5mem=subgroup.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration")
        subgroup_acceleration_5mem_items=list(subgroup_acceleration_5mem.items())
        print("\nAcceleration items_5members:", subgroup_acceleration_5mem_items)
        frequency=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Frequency")
        frequency_items=list(frequency.items())
        print("\nFrequency", frequency_items)
        magnitude=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Magnitude")
        magnitude_items=list(magnitude.items())
        print("\nMagnitude", magnitude_items)
        phase=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Phase")
        phase_items=list(magnitude.items())
        print("\nPhase", phase_items)
        
        
        #extract frequencies, magnitude, phase, uncertainties and all excitation parameters
        frequency_values=np.array(frequency.get("value"))
        magnitude_values=np.array(magnitude.get("value"))
        magnitude_uncertainties=np.array(magnitude.get("uncertainty"))
        phase_values=np.array(phase.get("value"))
        phase_uncertainties=np.array(phase.get("uncertainty"))
        excitation_freq_items=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Excitation_frequency")
        excitation_freq=np.array(excitation_freq_items.get("value"))
        excitation_amp_items=subgroup_acceleration_5mem.get("/RAWTRANSFERFUNCTION/"+sensor_index+"_MPU_9250/Acceleration/Acceleration/Excitation_amplitude")
        excitation_amp=np.array(excitation_amp_items.get("value"))
        excitation_amp_uncertainty=np.array(excitation_amp_items.get("uncertainty"))
        
        #join all necessary data in 2D array
        total_array=np.stack((frequency_values,magnitude_values,magnitude_uncertainties,phase_values, phase_uncertainties,excitation_freq,excitation_amp,excitation_amp_uncertainty), axis=1)
        print("\nArray dimensions:", total_array.shape)
        column_names=["Frequency [Hz]", r"$x_{M},$ [m s^-2/m s^-2]",r"$U_{M},$ [m s^-2/m s^-2]", r"$x_{\phi},$ [°]", r"$U_{\phi},$ [°]","Excitation_freq [Hz]",r"$x_{Aexcit},$ [m s^-2/m s^-2]",r"$U_{Aexcit},$ [m s^-2/m s^-2]"]
        whole_dataset=pd.DataFrame(total_array, columns=column_names)
        f.close()
        
        
        return whole_dataset

In [5]:
whole_dataset_PTB = extract_data('MPU9250PTB_v3.hdf5',"0x1fe40000")


Items in directory [('EXPERIMENTS', <HDF5 group "/EXPERIMENTS" (1 members)>), ('RAWDATA', <HDF5 group "/RAWDATA" (2 members)>), ('RAWTRANSFERFUNCTION', <HDF5 group "/RAWTRANSFERFUNCTION" (1 members)>), ('REFERENCEDATA', <HDF5 group "/REFERENCEDATA" (2 members)>)]

Items in reference [('0x1fe40000_MPU_9250', <HDF5 group "/RAWTRANSFERFUNCTION/0x1fe40000_MPU_9250" (1 members)>)]

0x1fe40000_MPU_9250 items: [('Acceleration', <HDF5 group "/RAWTRANSFERFUNCTION/0x1fe40000_MPU_9250/Acceleration" (1 members)>)]

Acceleration items: [('Acceleration', <HDF5 group "/RAWTRANSFERFUNCTION/0x1fe40000_MPU_9250/Acceleration/Acceleration" (10 members)>)]

Acceleration items_5members: [('DUT_Phase', <HDF5 group "/RAWTRANSFERFUNCTION/0x1fe40000_MPU_9250/Acceleration/Acceleration/DUT_Phase" (2 members)>), ('DUT_SNYNC_Phase', <HDF5 group "/RAWTRANSFERFUNCTION/0x1fe40000_MPU_9250/Acceleration/Acceleration/DUT_SNYNC_Phase" (2 members)>), ('DUT_amplitude', <HDF5 group "/RAWTRANSFERFUNCTION/0x1fe40000_MPU_9250/

In [6]:
whole_dataset_PTB.head(2)

,Frequency [Hz],"$x_{M},$ [m s^-2/m s^-2]","$U_{M},$ [m s^-2/m s^-2]","$x_{\phi},$ [°]","$U_{\phi},$ [°]",Excitation_freq [Hz],"$x_{Aexcit},$ [m s^-2/m s^-2]","$U_{Aexcit},$ [m s^-2/m s^-2]"
0,10.0,1.019554,0.002727,3.046909,0.002651,10.0,2.739,0.000548
1,12.5,1.019093,0.003008,3.022772,0.006248,12.5,2.744,0.000549


Phase data for PTB must be reverted:

In [7]:
whole_dataset_PTB[[r"$x_{\phi},$ [°]"]] = whole_dataset_PTB[[r"$x_{\phi},$ [°]"]]-np.pi

In [8]:
whole_dataset_PTB.head(2)

,Frequency [Hz],"$x_{M},$ [m s^-2/m s^-2]","$U_{M},$ [m s^-2/m s^-2]","$x_{\phi},$ [°]","$U_{\phi},$ [°]",Excitation_freq [Hz],"$x_{Aexcit},$ [m s^-2/m s^-2]","$U_{Aexcit},$ [m s^-2/m s^-2]"
0,10.0,1.019554,0.002727,-0.094683,0.002651,10.0,2.739,0.000548
1,12.5,1.019093,0.003008,-0.118820,0.006248,12.5,2.744,0.000549


In [9]:
whole_dataset_CEM = extract_data('MPU9250CEM_v3.hdf5',"0xbccb0000")


Items in directory [('EXPERIMENTS', <HDF5 group "/EXPERIMENTS" (1 members)>), ('RAWDATA', <HDF5 group "/RAWDATA" (2 members)>), ('RAWTRANSFERFUNCTION', <HDF5 group "/RAWTRANSFERFUNCTION" (1 members)>), ('REFERENCEDATA', <HDF5 group "/REFERENCEDATA" (2 members)>)]

Items in reference [('0xbccb0000_MPU_9250', <HDF5 group "/RAWTRANSFERFUNCTION/0xbccb0000_MPU_9250" (1 members)>)]

0xbccb0000_MPU_9250 items: [('Acceleration', <HDF5 group "/RAWTRANSFERFUNCTION/0xbccb0000_MPU_9250/Acceleration" (1 members)>)]

Acceleration items: [('Acceleration', <HDF5 group "/RAWTRANSFERFUNCTION/0xbccb0000_MPU_9250/Acceleration/Acceleration" (10 members)>)]

Acceleration items_5members: [('DUT_Phase', <HDF5 group "/RAWTRANSFERFUNCTION/0xbccb0000_MPU_9250/Acceleration/Acceleration/DUT_Phase" (2 members)>), ('DUT_SNYNC_Phase', <HDF5 group "/RAWTRANSFERFUNCTION/0xbccb0000_MPU_9250/Acceleration/Acceleration/DUT_SNYNC_Phase" (2 members)>), ('DUT_amplitude', <HDF5 group "/RAWTRANSFERFUNCTION/0xbccb0000_MPU_9250/

In [10]:
whole_dataset_CEM.head(2)

,Frequency [Hz],"$x_{M},$ [m s^-2/m s^-2]","$U_{M},$ [m s^-2/m s^-2]","$x_{\phi},$ [°]","$U_{\phi},$ [°]",Excitation_freq [Hz],"$x_{Aexcit},$ [m s^-2/m s^-2]","$U_{Aexcit},$ [m s^-2/m s^-2]"
0,80.0,0.973556,0.001220,-0.759661,0.003900,80.0,10.011,0.010011
1,250.0,0.642990,0.000894,-2.375413,0.063467,250.0,126.500,0.126500


Cycles in CEM's dataset start with 80.0 Hz and 250.0 Hz instead of 10.0 Hz. These starting points are deleted in order to compare the cycles in a range from 10.0 Hz and 250.Hz.

In [11]:
delete_rows=[]

for k in range(0,171,19):
    i=k
    j=k+1
    delete_rows.append(i)
    delete_rows.append(j)
whole_dataset_CEM_new=whole_dataset_CEM.drop(axis=0,index=delete_rows)

# 3.Data analysis

In [12]:
def split_data_by_frequencies(dataset):
    dict_of_frequencies=dict(iter(dataset.groupby('Frequency [Hz]')))
    return dict_of_frequencies
    #list_of_frequencies=np.array([10,12.5,16,20,25,31.5,40,46.7,50,53.3,63,80,100,125,160,200,250])

In [13]:
#check if all frequencies are the same
PTB_separated_by_freq=split_data_by_frequencies(whole_dataset_PTB)
CEM_separated_by_freq=split_data_by_frequencies(whole_dataset_CEM)
CEM_separated_by_freq_new=split_data_by_frequencies(whole_dataset_CEM_new)
print("Frequencies - PTB:",PTB_separated_by_freq.keys())
print("Frequencies - CEM:",CEM_separated_by_freq.keys())

Frequencies - PTB: dict_keys([10.0, 12.5, 16.0, 20.0, 25.0, 31.5, 40.0, 46.7, 50.0, 53.3, 63.0, 80.0, 100.0, 125.0, 160.0, 200.0, 250.0])
Frequencies - CEM: dict_keys([10.0, 12.5, 16.0, 20.0, 25.0, 31.5, 40.0, 46.7, 50.0, 53.3, 63.0, 80.0, 100.0, 125.0, 160.0, 200.0, 250.0])


In [14]:
PTB_separated_by_freq.get(10).head(1)

,Frequency [Hz],"$x_{M},$ [m s^-2/m s^-2]","$U_{M},$ [m s^-2/m s^-2]","$x_{\phi},$ [°]","$U_{\phi},$ [°]",Excitation_freq [Hz],"$x_{Aexcit},$ [m s^-2/m s^-2]","$U_{Aexcit},$ [m s^-2/m s^-2]"
0,10.0,1.019554,0.002727,-0.094683,0.002651,10.0,2.739,0.000548


In [15]:
CEM_separated_by_freq_new.get(10).head(1)

,Frequency [Hz],"$x_{M},$ [m s^-2/m s^-2]","$U_{M},$ [m s^-2/m s^-2]","$x_{\phi},$ [°]","$U_{\phi},$ [°]",Excitation_freq [Hz],"$x_{Aexcit},$ [m s^-2/m s^-2]","$U_{Aexcit},$ [m s^-2/m s^-2]"
2,10.0,1.016221,0.004506,-0.094899,0.023076,10.0,2.7543,0.002754


In [16]:
q_names=list(PTB_separated_by_freq.get(10).columns)

# 4. ANOVA for experiments performed at a given frequency

Ten measurement cycles for PTB and nine for CEM result with ten (or nine) experiments at each calibration point between 10.0 Hz and 250.0 Hz. Each experiment is described with:  ${x_{M}},{x_{\phi}}$ and assigned expanded uncertainties:
${U_{M}},{U_{\phi}}$. These mean values arise from the sine-fitting and conversion of time-domain signals to the frequency domain.  The ANOVA method is used to examine if the mean values of experiments at a given frequency are equal for each laboratory, PTB and CEM respectively. 

<br>In a general case, one-way ANOVA uses the following null and alternative hypotheses:

<br>H0 (a null hypothesis): μ1 = μ2 = μ3 = … = μk (all the population means are equal)
<br>H1 (a research hypothesis): at least one population mean is different from the rest.

There are three primary assumptions in ANOVA:

<br>*The responses for each factor level have a normal population distribution.*
<br>*These distributions have the same variance.*
<br>*The data are independent.*
<br>Violations to the first two that are not extreme can be considered not serious. The sampling distribution of the test statistic is fairly robust, especially as sample size increases and more so if the sample sizes for all factor levels are equal. If you conduct an ANOVA test, you should always try to keep the same sample sizes for each factor level.

If our samples have unequal variances (heteroscedasticity), on the other hand, it can affect the Type I error rate and lead to false positives. This is, basically, what equality of variances means.

<b>A general rule of thumb for equal variances is to compare the smallest and largest sample standard deviations. This is much like the rule of thumb for equal variances for the test for independent means. If the ratio of these two sample standard deviations falls within 0.5 to 2, then it may be that the assumption is not violated.<b>

## 4.1 Test for the equality of variances

In [17]:
def rule_of_thumb_ANOVA(dictionary, index):  
    ratio_mag=np.empty((len(dictionary.values())))
    ratio_ph=np.empty((len(dictionary.values())))
    ratio_ex=np.empty((len(dictionary.values())))
    for val,f in zip (dictionary.values(),range(len(dictionary.values()))):
            min_mag=min(val[r"$U_{M},$ [m s^-2/m s^-2]"].values/2)
            max_mag=max(val[r"$U_{M},$ [m s^-2/m s^-2]"].values/2)
            ratio_mag[f]=max_mag/min_mag

            min_ph=min(val[r"$U_{\phi},$ [°]"].values/2)
            max_ph=max(val[r"$U_{\phi},$ [°]"].values/2)
            ratio_ph[f]=max_ph/min_ph

            min_ex=min(val[r"$U_{Aexcit},$ [m s^-2/m s^-2]"].values/2)
            max_ex=max(val[r"$U_{Aexcit},$ [m s^-2/m s^-2]"].values/2)
            ratio_ex[f]=max_ex/min_ex
    
    ratios = {'Magnitude' : pd.Series(ratio_mag,index =index),
              'Phase' : pd.Series(ratio_ph,index =index),
              'Excitation amplitude' : pd.Series(ratio_ex,index =index),     
             }
    ratios=pd.DataFrame(ratios, index=index)  
    return ratios#.style.applymap(lambda x: 'background-color : red' if x>2 else 'background-color : green')
 #red - variances are not equal.The assumption might be violated
# green - variances are equal. The assumption is not violated

In [18]:
list_of_freq=[10,12.5,16,20,25,31.5,40,46.7,50,53.3,63,80,100,125,160,200,250]
rule_PTB=rule_of_thumb_ANOVA(PTB_separated_by_freq,list_of_freq)
rule_CEM=rule_of_thumb_ANOVA(CEM_separated_by_freq_new,list_of_freq)

# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()


with widget1:
    display.display(rule_PTB.style.set_caption('PTB').applymap(lambda x: 'background-color : red' if x>2 else 'background-color : green'))

with widget2:
    display.display(rule_CEM.style.set_caption('CEM').applymap(lambda x: 'background-color : red' if x>2 else 'background-color : green'))



# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2], layout=box_layout)

# render hbox
hbox

The test for the equality of variances shows that distributions do not have the same variance at frequencies with the highest magnitude uncertainties (10.0 Hz - 16.0 Hz) for PTB and in a range (50.0 - 250.0 Hz) (Reference:1_Introduction_data_analysis_.ipynb - 3.1.). The interesting point is 31.5 Hz, where the uncertainty of magnitude is moderate when compared to those at other frequencies.

Phase observation for PTB shows that the test for equality of variances is not satisfied in a range (31.5 - 53.3) Hz, where the highest uncertainties were noticed  and at 80.0 Hz, where the uncertainty is moderate when compared to those at other frequencies.


Magnitude observation for CEM shows that the test for equality of variances is not satisfied in a range (63.0-250.0) Hz, which covers the range (63.0-125.0) Hz, where the highest uncertainties were detected. Sporadically, the test requirement is not satisfied at 10.0 Hz, 25.0 Hz, 46.7 Hz and 50.0 Hz, where expanded uncertainty is higher that 0.003 m s^-2/m s^-2.
Test for the equality of variances is stasfied for all phase values.

This test raise awareness in potential violations in the results which will be provided further by the ANOVA method.


If it is assumed that each experiment is based on at least 30 single values, then the normal distribution can be proposed:
$$X \hookrightarrow  \mathcal{N}(x_{M},\,\sigma _{M}^{2})$$
where $\sigma _{M}$ is calculated as   $\frac{U _{M}}{2}$.

The same approach refers to the phase values: 
$$X \hookrightarrow  \mathcal{N}(x_{\phi},\,\sigma _{\phi}^{2})$$
where $\sigma _{\phi}$ is calculated as   $\frac{U _{\phi}}{2}$.
Now, the task of ANOVA for the magnitude values is to set the null- and research hypothesis and to examine if the null-hypothesis can be rejected or not is to examine whether $x_{M1} = x_{M2}=.....x_{Mi}$
<br> H0:  $x_{M1} = x_{M2}=.....x_{Mi}$
<br>H1 (a research hypothesis): at least one $x_{Mi}$ is different from the rest.

The task of ANOVA for the phase  values is to set the null- and research hypothesis and to examine if the null-hypothesis can be rejected or not is to examine whether $x_{\phi 1} = x_{\phi 2}=.....x_{\phi i}$
<br> H0:  $x_{\phi 1} = x_{\phi 2}=.....x_{\phi i}$
<br>H1 (a research hypothesis): at least one $x_{\phi i}$ is different from the rest.

In both cases, *i* refers to the number of experiments at a given frequency (*i*=10 for PTB and *i*=9 for CEM).

<br> If the p-value of the ANOVA calculation is higher that 0.05, the null hypothesis cannot be rejected. [4] If the p-value is less than 0.05, we reject the null hypothesis that there's no difference between the means.
<br>*NOTE: ANOVA method at this step is based on sampling because for each experiment only the mean value and standard deviation of quantities of interest are known, while the number of single values contained in the each experiment is not known. However, it is not sure whether the sampling is an applicable option for measurement procedure by the acceleration sensor.It is also noticable that the results are affected by the number of samples involved.*

In [19]:
def ANOVA_through_experiments(dictionary,index,lab):
 
    p=np.empty((len(dictionary.values())))

    for val,f in zip (dictionary.values(),range(len(dictionary.values()))):
        group=[None]*val.shape[0]
        for item in range(val.shape[0]):
            random1=np.random.normal(val[r"$x_{M},$ [m s^-2/m s^-2]"].values[item], val[r"$U_{M},$ [m s^-2/m s^-2]"].values[item]/2, 30)
            group[item]=random1
            
        if lab=="PTB": 
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8],group[9]) 
        elif lab=="CEM":
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8])
        p[f]=pVal
     
    df=pd.DataFrame(p,columns=["p-value (Magnitude) "], index=index) 

          
    for val,f in zip (dictionary.values(),range(len(dictionary.values()))):
        group=[None]*val.shape[0]
        for item in range(val.shape[0]):
            random1=np.random.normal(val[r"$x_{\phi},$ [°]"].values[item], val[r"$U_{\phi},$ [°]"].values[item]/2, 30)
            group[item]=random1
        if lab=="PTB": 
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8],group[9])
            p[f]=pVal
        elif lab=="CEM":
            F_statistic, pVal = stats.f_oneway(group[0],group[1],group[2],group[3],group[4],group[5],group[6],group[7],group[8])
        p[f]=pVal
  
    df["p-value (Phase) "]=p    
        
    return df#.style.applymap(lambda x: 'background-color : yellow' if x<0.05 else 'background-color : green')

 #yellow - the null hypothesis can be rejected. 
# green - the null hypothesis cannot be rejected. 

In [20]:
p_ANOVA_PTB,p_ANOVA_CEM=ANOVA_through_experiments(PTB_separated_by_freq,list_of_freq,"PTB"),ANOVA_through_experiments(CEM_separated_by_freq_new,list_of_freq,"CEM")

In [21]:
# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()

with widget1:
    display.display(p_ANOVA_PTB.style.set_caption('PTB').applymap(lambda x: 'background-color : yellow' if x<0.05 else 'background-color : green'))

with widget2:
    display.display(p_ANOVA_CEM.style.set_caption('CEM').applymap(lambda x: 'background-color : yellow' if x<0.05 else 'background-color : green'))



# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2], layout=box_layout)

# render hbox
hbox

# References

[1] https://www.investopedia.com/terms/c/coefficientofvariation.asp
<br>[2] https://en.wikipedia.org/wiki/Weighted_arithmetic_mean
<br>[3] https://en.wikipedia.org/wiki/Effective_sample_size
<br>[4] https://online.stat.psu.edu/stat500/lesson/10/10.2/10.2.1
<br>[5] https://www.marsja.se/levenes-bartletts-test-of-equality-homogeneity-of-variance-in-python/